# Dataset creation

## Installation

In [3]:
!pip install -q llama-cpp-python textdescriptives argilla==1.18 transformers datasets langdetect


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
!CT_METAL=1 pip install -q ctransformers --no-binary ctransformers

DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# from ctransformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral")

# print(llm("AI is going to"))

## Imports

In [137]:
import textdescriptives as td
from datasets import load_dataset
import re
import spacy
from langdetect import detect
import argilla as rg
import numpy as np

## Pre-processing

End-to-end workflow to create a dataset in Argilla with text measurements as metadata.
This aids in quickly identifying and improving potential dataset issues.

### Dataset creation

At first, we need to create a dataset in Argilla. This can either be done by loading a previous created dataset or by creating a new one. In order to avoid duplication, we will check if the dataset already exists.

In [18]:
try:
    ds_local = rg.FeedbackDataset.for_supervised_fine_tuning(context=True, use_markdown=True, guidelines=None)
    ds_local.questions.extend([
        rg.RatingQuestion(
            name="prompt-quality", 
            title="Prompt Quality",
            values=list(range(1, 8)), 
            description="How would you rate the quality of the prompt?",
        ),
        rg.LabelQuestion(
            name="prompt-intent", 
            title="Prompt Intent",
            labels=["generation", "rewrite", "extract", "closed-qa", "open-qa", "classification", "summarization", "brainstorming", "chat", "code", "other"], 
            description="What is the intent of the prompt?"
        ),
        rg.MultiLabelQuestion(
            name="prompt-toxicity", 
            title="Prompt Toxicity",
            labels=["illegal", "harmfull", "unqualified advice"], 
            description="What are the toxicities in the prompt (if any)?",
            required=False
        )
    ])
    ds_remote = ds_local.push_to_argilla("sharegpt")
except Exception as e:
    ds_remote = rg.FeedbackDataset.from_argilla("sharegpt")
ds_remote

RemoteFeedbackDataset(
   id=830085b1-274e-4f69-a463-d681faca0377
   name=sharegpt
   workspace=Workspace(id=e41766d9-89eb-4497-afce-1d96c7ff5f51, name=argilla, inserted_at=2023-10-28 11:26:52.619081, updated_at=2023-10-28 11:26:52.619081)
   url=http://localhost:6900/dataset/830085b1-274e-4f69-a463-d681faca0377/annotation-mode
   fields=[RemoteTextField(id=UUID('223f85ab-765a-42be-aed7-95bdbf5498e5'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('fca18ae5-2cd7-4b2b-bffa-27c13bf75e70'), client=None, name='context', title='Context', required=False, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('92f87e74-16b2-42a9-a18c-4ff0b38c9606'), client=None, name='response', title='Response', description=None, required=True, type='text', use_markdown=True), RemoteRatingQuestion(id=UUID('f2c1403a-cb1a-486d-8c56-5dd96f3cda0a'), client=None, name='prompt-quality', title='Prompt Quality', description=None

#### Configure the metadata-properties

Next we will be using `text-descriptives` to configure the metadata-properties. This will be used to add and updat relevant metadata-properties to the dataset. Because `text-descriptives` doesn't provide any programmatic interface with the metrics-groups and their sub-metrics, we will run the computation on the an example text and use the results to configure the metadata-properties.

In [112]:
metric_group = ["descriptive_stats"]
relevant_subgroups = []
df_metrics = td.extract_metrics(
    text=["this is an example prompt"], 
    lang="en", 
    metrics=metric_group,
    spacy_model="en_core_web_sm"
).drop(columns=["text"] + relevant_subgroups if relevant_subgroups else ["text"])
df_metrics.columns

ℹ Both a spacy model and a language were provided. Will use the spacy
model and ignore language.


Index(['token_length_mean', 'token_length_median', 'token_length_std',
       'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
       'syllables_per_token_mean', 'syllables_per_token_median',
       'syllables_per_token_std', 'n_tokens', 'n_unique_tokens',
       'proportion_unique_tokens', 'n_characters', 'n_sentences'],
      dtype='object')

Next, we will be working on converting the `text-descriptives` output to a format that can be used to configure the metadata-properties for our supported types: `TermsMetadataProperty`, `IntegerMetadataProperty` and `FloatMetadataProperty`. Note that we are also applying some subjective formatting choices to ensure that the metadata-properties are easy to read and understand.

In [113]:
def clean_column_name(col_name):
    """Clean a column name to fit a specific regex pattern."""
    col_name = col_name.lower()  # Convert to lowercase
    col_name = re.sub(r'[^a-z0-9_]', '_', col_name)  # Replace non-alphanumeric characters with underscores
    return col_name

def create_metadata_properties(df, prefix):
    """Generate metadata properties based on dataframe columns and data types."""
    properties = []
    for col, dtype in df.dtypes.items():
        name = f"{prefix}_{clean_column_name(col)}"
        title = name.replace('_', ' ').title()

        if dtype == 'object':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        elif dtype == 'int64':
            prop = rg.IntegerMetadataProperty(name=name, title=title)
        elif dtype == 'float64':
            prop = rg.FloatMetadataProperty(name=name, title=title)
        elif dtype == 'bool':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        else:
            print(f"Unhandled data type for column {col}: {dtype}")
            continue
        properties.append(prop)
    return properties

metadata_properties = []
metadata_properties += create_metadata_properties(df_metrics, 'prompt')
metadata_properties += create_metadata_properties(df_metrics, 'response')
for metadata_property in metadata_properties:
    try:
        field = ds_remote.metadata_property_by_name(metadata_property.name)
        if not field:
            ds_remote.add_metadata_property(metadata_property)
    except (KeyError, ValueError) as e:
        ds_remote.add_metadata_property(metadata_property)        
ds_remote.metadata_properties

[RemoteFloatMetadataProperty(id=UUID('9691b808-c85e-4248-98bd-89bc2a81b2eb'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_mean', title='Response Token Length Mean', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('949d3bf5-a8ec-47c8-a4ce-249a9ee4a922'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_median', title='Response Token Length Median', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('43c7f5a7-e856-4794-9da8-6f89848a3a01'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_std', title='Response Token Length Std', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('4dcca678-229c-43e8-b0d0-9f5fee2ec902'), client=<httpx.Client object at 0x11105d3d0>, name='response_sentence_length_mean', title='Response Sentence Length Mean', visible_for_annotators=T

### Data collection

In [140]:
dataset = load_dataset("totally-not-an-llm/sharegpt-hyperfiltered-3k", split="train[:10]")
dataset = dataset.filter(lambda x: x["conversations"][0]["from"] == "human")
dataset = dataset.map(lambda x: {"prompt": x["conversations"][0]["value"], "response": x["conversations"][1]["value"]})

# Extract metrics
spacy_model = "en_core_web_md" # we need a model with vectors
df_prompt = td.extract_metrics(text=dataset["prompt"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])
df_response = td.extract_metrics(text=dataset["response"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])

# Identify integer and boolean columns for prompts and responses
int_cols_prompts = df_prompt.select_dtypes(include=['int64']).columns.tolist()
bool_cols_prompts = df_prompt.select_dtypes(include=['boolean']).columns.tolist()

int_cols_responses = df_response.select_dtypes(include=['int64']).columns.tolist()
bool_cols_responses = df_response.select_dtypes(include=['boolean']).columns.tolist()

# Combine column lists for prompts and responses
int_cols = list(set(int_cols_prompts + int_cols_responses))
bool_cols = list(set(bool_cols_prompts + bool_cols_responses))
int_cols, bool_cols

(['n_sentences', 'n_characters', 'n_unique_tokens', 'n_tokens'], [])

Next, we will be casting the `numpy`-datatypes to basic Python built-in datatypes. This is required because the Argilla client doesn't support `numpy`-datatypes.

In [142]:
# --- Functions ---
def cast_to_python_types(df):
    """
    Convert integer and boolean columns to Python native types.
    """
    for column in df.columns:
        df[column].fillna(0, inplace=True)
        if df[column].dtype == bool:
            df[column] = df[column].astype(str)
        elif df[column].dtype == np.int64:
            df[column] = df[column].astype(int)
        elif df[column].dtype == np.float64:
            df[column] = df[column].astype(float)
        else:
            print(f"Unhandled data type for column {column}: {df[column].dtype}")
    return df

df_prompt = cast_to_python_types(df_prompt)
df_response = cast_to_python_types(df_response)

Lastly, we will loop through the Hugging Face dataset, add the metadata-properties and update the Argilla dataset with the new records.

In [143]:
# Prepare feedback records with metadata and suggestions
records = []

cols_with_values_other_than_zeros_or_nan_prompt = df_prompt.columns[~(df_prompt.fillna(0) == 0).all() & ~df_prompt.isnull().any()].tolist()
cols_with_values_other_than_zeros_or_nan_response = df_response.columns[~(df_response.fillna(0) == 0).all() & ~df_response.isnull().any()].tolist()


for i, record in enumerate(dataset):
    # Prepare metadata for prompts
    metadata_prompts = {f"prompt_{col}": value for col, value in df_prompt[cols_with_values_other_than_zeros_or_nan_prompt].iloc[i].items()}
    # Prepare metadata for responses
    metadata_response = {f"response_{col}": value for col, value in df_response[cols_with_values_other_than_zeros_or_nan_response].iloc[i].items()}

    # Explicitly cast integers using Python's native int type
    for col in int_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = int(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = int(metadata_response[f"response_{col}"])

    # Convert booleans to strings using Python's native str type
    for col in bool_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = str(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = str(metadata_response[f"response_{col}"])

    # Combine both metadata dictionaries into one
    metadata = {**metadata_prompts, **metadata_response}
    record = rg.FeedbackRecord(
        fields={"prompt": record["prompt"]},
        metadata=metadata,
        suggestions=[{"question_name": "response", "value": record["response"]}]
    )
    records.append(record)

# Add records to the dataset and push to Argilla
ds_remote.add_records(records)

Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00, 12.69it/s]
